# Proyecto - Parte 1 
## SID 2025-20

| Estudiante | Codigo | Maestria |
| :------- | :------: | -------: |
| David Rodriguez | 201523585| MATI |
| Jhony Donosso| 202322580 | MATI |
| Nicolas Medina | 202322627 | MATI |

a continuacion se detalla la solución planteada para la primera parte del proyecto

## Importar librerias/Funciones

In [0]:
from pyspark.sql import functions as f
import pyspark.sql as sql
from pyspark.sql.functions import explode, split, lower, col
from pyspark.sql.window import Window
from pyspark.sql.types import DecimalType

## Lectura de datos

In [0]:
df_secop = spark.read.format("json").load("wasbs://sid@uniandesyjt.blob.core.windows.net/secop/")

df_secop.printSchema()

In [0]:
df_bpin = spark.read.option("header", "true").format("csv").load("wasbs://sid@uniandesyjt.blob.core.windows.net/bpin/")

df_bpin.printSchema()

In [0]:
display(df_secop)

In [0]:
display(df_bpin)

# Punto 1:
Lea los dos conjuntos de datos. Escriba el código para que la lectura sea lo más rápida
posible y justifique su respuesta. (9%)

# Punto 2:
Identifique los 10 proveedores que han tenido el mayor valor de contratos durante el año
2024. (13%)

In [0]:
top_10_proveedores_2024 = (
    df_secop
    .filter(f.year(f.col("fecha_de_firma")) == 2024)
    .groupBy("proveedor_adjudicado")
    .agg(f.sum("valor_del_contrato").alias("total_valor_contrato"))
    .orderBy(f.desc("total_valor_contrato"))
    .limit(10)
)

display(top_10_proveedores_2024)

#Punto 3:
Identifique los 10 Proyectos de inversión que han tenido el mayor valor de contratos sin
pagar (valor del contrato menos el valor pagado). La respuesta debe incluir el nombre de
los proyectos y el valor sin pagar. (13%)


In [0]:
df_bpin_csp=df_bpin.withColumn("ValorContratosSinPagar",col("ValorTotalProyecto").cast("double")-col("ValorPagoProyecto").cast("double"))
top_10_contratos_sin_pagar_df=df_bpin_csp.groupBy("NombreProyecto").agg(f.sum("ValorContratosSinPagar").alias("ValorContratosSinPagar")).orderBy(f.desc("ValorContratosSinPagar")).limit(10)
display(top_10_contratos_sin_pagar_df)

In [0]:
df_secop_std = f.coalesce(f.col("valor_pagado").cast("string"), f.lit("0")) #Estandarizamos los valores sin null por 0
#Limpiamos digitos y signos
df_secop_std = f.regexp_replace(df_secop_std, r"[^0-9]", "") #Quitamos espacios y $
df_secop_std = f.regexp_replace(df_secop_std, ",", "") #Quitamos la coma
df_secop_std = df_secop_std.cast(DecimalType(20,2))

#Generamos un df con la info necesaria
df_secop_norm_p3 = (
    df_secop
        .withColumn("Bpin", f.trim(f.col("código_bpin"))).alias("Bpin")
        .withColumn("valor_del_contrato", f.col("valor_del_contrato").cast(DecimalType(20,2)))
        .withColumn("valor_pagado", df_secop_std)
        .select("Bpin", "valor_del_contrato", "valor_pagado")
)
#Sacamos el valor total sin pagar
top_10_contratos_sin_pagar_df = (
    df_secop_norm_p3.groupBy("Bpin")
    .agg(
        f.sum("valor_del_contrato").alias("ValorTotalProyecto"),
        f.sum(f.coalesce("valor_pagado", f.lit(0))).alias("ValorTotalPagado")
    )
    .withColumn("ValorSinPagar", f.col("ValorTotalProyecto") - f.col("ValorTotalPagado"))
)
#traemos el nombre del proyecto de df-bpin
top_10_join = (
    top_10_contratos_sin_pagar_df.join(df_bpin.select("Bpin", "NombreProyecto"), on="Bpin", how="left")
    .orderBy(f.col("ValorSinPagar").desc_nulls_last())
    .limit(10)
    .select("Bpin","ValorSinPagar","NombreProyecto")
)
display(top_10_join)


#Punto 4:
Identifique para cada año el top 5 de los proveedores con mayor valor de contratos. La
respuesta debe incluir el nombre de los proveedores, el valor de sus contratos y su
posición en el top. (13%)

In [0]:
#Agrupamos por año y proveedor para realizar la agregación de los contratos calculando total valor contratos
top_proveedores=(
    df_secop.groupBy("anno_firma", "proveedor_adjudicado")
    .agg(f.sum("valor_del_contrato").alias("total_valor_contratos"))
)
#Establecemos una ventana  por año firma, ordenamos por valor total contratos
window = Window.partitionBy("anno_firma").orderBy(f.desc("total_valor_contratos"))
#Usamos row number para asignar un ranking a cada proveedor
top5_proveedores=(
    top_proveedores.withColumn("Ranking", f.row_number().over(window))
    .filter(f.col("Ranking") <= 5)
)
display(top5_proveedores)


#Punto 5:
Identifique el número de proveedores que firmaron contratos en el 2024 y NO firmaron
contratos en el 2020. (13%)

#Punto 6:
Calcule el valor promedio de los contratos de “Prestación de servicios” y una
aproximación de la mediana (el percentil 50). Construya esta respuesta de manera que se
minimice el tiempo de cálculo, aunque se sacrifique precisión. (13%)


#Punto 7:
Cuál fue el año en el que firmaron contratos un mayor número de proveedores distintos.
Construya esta respuesta de manera que se minimice el tiempo de cálculo, aunque se
sacrifique precisión. (13%)

#Punto 8:
Cuáles son las 20 palabras más comunes en los objetos contractuales del 20% de los
contratos más altos del 2020. Para esta respuesta omita stopwords
(https://en.wikipedia.org/wiki/Stop_word, https://pypi.org/project/stop-words/),
puntuaciones y no diferencie entre mayúsculas y minúsculas. (13%)